# Prototype

# Part 1 : Running a Classifier

The initial goal of this prototype is to run a classifier on the provided.

The classifier is also only comparing itself to HS

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

path = '../data/hateval2019_en_dev_clean.csv'
testset = pd.read_csv(path)

testset.head(5)

,Unnamed: 0,id,text,HS,TR,AG
0,0,18201,I swear I’m getting to places just in the nick...,0,0,0
1,1,18202,I’m an immigrant — and Trump is right on immig...,0,0,0
2,2,18203,#IllegalImmigrants #IllegalAliens #ElectoralSy...,1,0,1
3,3,18204,@USER We have our own invasion issues with Mex...,1,0,1
4,4,18205,Worker Charged With Sexually Molesting Eight C...,0,0,0


In [2]:
testset.shape

(1000, 6)

In [3]:
testset.head(10)

,Unnamed: 0,id,text,HS,TR,AG
0,0,18201,I swear I’m getting to places just in the nick...,0,0,0
1,1,18202,I’m an immigrant — and Trump is right on immig...,0,0,0
2,2,18203,#IllegalImmigrants #IllegalAliens #ElectoralSy...,1,0,1
3,3,18204,@USER We have our own invasion issues with Mex...,1,0,1
4,4,18205,Worker Charged With Sexually Molesting Eight C...,0,0,0
5,5,18206,"UN picks refugees for UK, picks mostly Syrian ...",0,0,0
6,6,18207,@USER Delhi will be much better under Prez Rul...,0,0,0
7,7,18208,Global Compact on Migration moves forward http...,0,0,0
8,8,18209,"Violence toward tourists in Barcelona, Spain -...",0,0,0
9,9,18210,Immigration decline costing UK economy billion...,0,0,0


As can be seen from the data here, the mentions in the original tweet is still present within this dataset.

This should be scrubbed for data privacy

In [4]:
testset.HS.value_counts()

0    573
1    427
Name: HS, dtype: int64

In [5]:
testset.TR.value_counts()

0    781
1    219
Name: TR, dtype: int64

In [6]:
testset.AG.value_counts()

0    796
1    204
Name: AG, dtype: int64

In [7]:
x = testset.text
y = testset.HS

print(x.shape)
print(y.shape)

(1000,)
(1000,)


In [8]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y, random_state=1)

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(750,)
(250,)
(750,)
(250,)


In [9]:
vect = CountVectorizer(stop_words='english', ngram_range=(1, 2), min_df=2)

x_train_dtm = vect.fit_transform(x_train)
x_train_dtm

<750x1651 sparse matrix of type '<class 'numpy.int64'>'
	with 8463 stored elements in Compressed Sparse Row format>

In [10]:
x_test_dtm = vect.transform(x_test)
x_test_dtm

<250x1651 sparse matrix of type '<class 'numpy.int64'>'
	with 2271 stored elements in Compressed Sparse Row format>

In [11]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()

%time nb.fit(x_train_dtm, y_train)

Wall time: 2.03 ms


MultinomialNB()

In [12]:
y_pred_class = nb.predict(x_test_dtm)

In [13]:
from sklearn import metrics

nb_acc = metrics.accuracy_score(y_test, y_pred_class)
nb_acc

0.704

In [14]:
metrics.confusion_matrix(y_test, y_pred_class)

array([[101,  43],
       [ 31,  75]], dtype=int64)

In [15]:
x_test[(y_pred_class==1) & (y_test==0)]

507             @USER Shut the fuck up you pussy boy!!!!
818    you gotta be a fucking BITCH to play with a gi...
810                   Screw up your face and skank music
968                      @USER you pussy ass bitch. stfu
702    i was also clenching my jaw and further grindi...
910                         @USER Skank man .. you got!!
691    @USER @USER @USER @USER True that Morgan!!!!If...
94     Ontario concerns were not listened at immigrat...
577    Cory Booker thinks he's the most hysterical wo...
41     Germany try to deport criminel Afghanistan ref...
995              @USER you unfollowed me? Fuck you pussy
885    nigga, im pretty sure you dont get pussy, just...
677    Mr Fucking Graham Shut the Fuck up about fucki...
822    @USER And you deleted your tweet you pussy shu...
812    "@USER YA Allah!This filthy skank needs a ""ca...
17     Texas woman, 21, dies after falling from movin...
191    Illegal aliens are expensive:  https://t.co/63...
638    @USER Hysterical @USER D

In [16]:
x_test[(y_pred_class==0) & (y_test==1)]

368    @USER If the refugees dont get food and water ...
446    @USER Hear hear May bows down and grovels plea...
374    I do not understand the minds of some. If the ...
614    @USER @USER Yes yes Ann, lets continue to perp...
241    #Refugees not welcome can’t be deported, becau...
654    @USER @USER @USER Yes so obtuse every other co...
189    Use foreign aid budget to help illegal migrant...
335    EU: How to Stop Mass-migration from Africa? Br...
538    For crying out loud, Ivanka! STFU you ignorant...
479    Germany now worried that the surge of African ...
175    @USER Lot of illegal Muslim immigrants r seen ...
794    Nah yall stay in the kitchen and make bariis a...
2      #IllegalImmigrants #IllegalAliens #ElectoralSy...
73     Exclusive: Sanctioned Libya coastguard command...
111    Corbyn still maintains Blair's New Labour whic...
555    #WomensMarch collected hundreds of hysterical ...
552    Brown Nose Nikki, You were silent when Donnie ...
887    I usually dont hate peop

In [17]:
metrics.roc_auc_score(y_test, y_pred_prob)

NameError: name 'y_pred_prob' is not defined

# Part 2 : Cleaning Up Data

As can be seen in the earlier prototype, the tweet still contains its original mention.

This needs to be removed